In [377]:
import numpy as np

In [378]:
import os

In [379]:
import matplotlib.pyplot as plt

In [380]:
def stacked_images_array(path):
    
    list_of_folders = os.listdir(path)
    
    list_of_images = []

    parent_path = path

    for each_folder in list_of_folders:
    
        folder_path = os.path.join(parent_path,each_folder)
    
        list_of_images.extend(map(lambda x: plt.imread(os.path.join(folder_path,x)).reshape(1,1024),os.listdir(folder_path)))
        
    return list_of_images,list_of_folders

In [381]:
list_of_images,list_of_folders = stacked_images_array("./Datasets/DevanagariHandwrittenCharacterDataset/Train")

In [382]:
training_data = np.concatenate(list_of_images)

In [383]:
training_data.shape

(78200, 1024)

In [384]:
unique_labels = np.arange(0,len(list_of_folders))

In [385]:
unique_labels

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45])

In [386]:
labels = []

labels.extend(map(lambda x: x*np.ones((1700,1)),unique_labels))

In [387]:
labels = np.concatenate(labels)

In [388]:
labels.shape

(78200, 1)

In [389]:
import pandas as pd

In [390]:
training_data = pd.DataFrame(data=training_data)

In [391]:
training_data['labels'] = labels

In [392]:
training_data.head()

,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,labels
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [393]:
classes_cov_mats = []

classes_cov_mats.extend(map(lambda x: np.array(training_data[training_data['labels'] == x].iloc[:,0:1024].cov()),unique_labels))

In [394]:
len(classes_cov_mats)

46

In [395]:
np.linalg.det(classes_cov_mats[0])

0.0

In [396]:
def regularized_matrices(classes_cov_mats):

    alpha = np.random.uniform(0,1)

    gamma = np.random.uniform(0,1)

    rda_cov_mats1 = []
    
    pooled_cov_mat = (1700-1)*sum(classes_cov_mats)/(training_data.shape[0] - len(list_of_folders))

    rda_cov_mats1.extend(map(lambda x: ((alpha*pooled_cov_mat) + ((1-alpha)*x)),classes_cov_mats))

    rda_mats1_diag_avgs = []

    rda_mats1_diag_avgs.extend(map(lambda x: np.mean(np.diag(x)),rda_cov_mats1))

    rda_cov_mats2 = []

    rda_cov_mats2.extend(map(lambda x: ((gamma*rda_cov_mats1[x]) + ((1-gamma)*rda_mats1_diag_avgs[x]*np.eye(rda_cov_mats1[0].shape[0],rda_cov_mats1[0].shape[0]))),np.arange(0,len(list_of_folders))))
    
    return rda_cov_mats2

In [397]:
rda_matrices = regularized_matrices(classes_cov_mats)

In [398]:
np.linalg.det(rda_matrices[0])

0.0

In [399]:
def PCA(cov,preserved_variance):
    
    svd_factorized_mat = np.linalg.svd(cov)
    
    eig_vals = svd_factorized_mat[1]
    
    eig_vecs = svd_factorized_mat[0]
    
    eig_vals_total = np.sum(eig_vals)
    
    eig_vals_sum = 0
    
    indices = []
    
    for i in range(0,eig_vals.shape[0]):
        
        if eig_vals_sum/eig_vals_total > preserved_variance:
            
            break
            
        eig_vals_sum += eig_vals[i]
        
        indices.append(i)
        
    return eig_vecs[:,indices]

In [400]:
preserved_variance = 0.97

In [401]:
training_data_cov = np.array(training_data.iloc[:,0:1024].cov())

In [402]:
eig_vecs_mat = PCA(training_data_cov,preserved_variance)

In [403]:
training_data_mat = np.array(training_data.iloc[:,0:1024])

In [404]:
eig_vecs_mat.shape

(1024, 239)

In [405]:
pca_reduced_data = np.matmul(training_data_mat,eig_vecs_mat)

In [406]:
pca_reduced_data.shape

(78200, 239)

In [407]:
classes_cov_mats_reduced = []

classes_cov_mats_reduced.extend(map(lambda x: np.cov(pca_reduced_data[(1700*x):(1700*(x+1)),:],rowvar=False),unique_labels))

In [408]:
len(classes_cov_mats_reduced)

46

In [409]:
correct_counts = []

accuracies = []

In [436]:
pca_reduced_rda_matrices = regularized_matrices(classes_cov_mats_reduced)

In [412]:
import scipy.stats as s

In [413]:
list_of_images,list_of_folders = stacked_images_array("./Datasets/DevanagariHandwrittenCharacterDataset/Test")

In [414]:
testing_data = np.array(list_of_images)

In [415]:
testing_data = testing_data.reshape(testing_data.shape[0],testing_data.shape[2])

In [416]:
pca_reduced_testing_data = np.matmul(testing_data,eig_vecs_mat)

In [417]:
pca_reduced_testing_data.shape

(13800, 239)

In [418]:
class_mean_vecs = []

class_mean_vecs.extend(map(lambda x: np.mean(pca_reduced_data[1700*x:1700*(x+1),:],axis=0),np.arange(0,len(list_of_folders))))

In [419]:
len(class_mean_vecs)

46

In [420]:
len(pca_reduced_rda_matrices)

46

In [421]:
def post_prob(class_means,class_cov):
    
    p_class_on_x = s.multivariate_normal.pdf(pca_reduced_testing_data,class_means,class_cov)
    
    p_class_on_x = p_class_on_x.reshape(p_class_on_x.shape[0],1)
    
    return p_class_on_x

In [437]:
post_probs = []

post_probs.extend(map(lambda x: post_prob(class_mean_vecs[x],pca_reduced_rda_matrices[x]),np.arange(0,len(list_of_folders))))

In [438]:
posterior_probabilities = np.concatenate(post_probs,axis=1)

In [439]:
posterior_probabilities.shape

(13800, 46)

In [440]:
predicted_classes = np.argmax(posterior_probabilities,axis=1)

In [441]:
predicted_classes.shape

(13800,)

In [442]:
predicted_classes = predicted_classes.reshape(predicted_classes.shape[0],1)

In [443]:
actual_classes = []

actual_classes.extend(map(lambda x: x*np.ones((300,1)),np.arange(0,len(list_of_folders))))

In [444]:
actual_classes = np.concatenate(actual_classes)

In [445]:
actual_classes.shape

(13800, 1)

In [446]:
correct_count = np.count_nonzero(np.equal(predicted_classes,actual_classes))

In [447]:
correct_counts.append(correct_count)

In [448]:
correct_counts

[8469, 12306]

In [449]:
accuracies.append((correct_count/actual_classes.shape[0])*100)

In [450]:
accuracies

[61.36956521739131, 89.17391304347827]